In [6]:
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as pyo

In [2]:
read_csv = lambda path: pd.read_csv(path, parse_dates=[0], index_col=0)
metrics = [
    read_csv('../layers/application/content-locality/content-locality.csv'),
    read_csv('../layers/protocol/ipv6-transition/ipv6-transition.csv'),
    read_csv('../layers/physical/undersea-cables/undersea-cables.csv'),
]

In [3]:
merged_df = metrics[0]
for metric in metrics[1:]:
     merged_df = merged_df.merge(metric, left_index=True, right_index=True, how="outer")

merged_df = merged_df\
.sort_index()\
.fillna(method='ffill')

ifi = merged_df.dropna().mean(axis=1)

merged_df['IFI'] = ifi

df = merged_df.reset_index()


In [11]:
# TODO - create a naming scheme at the metric level
df = df.rename(columns={
    'jaccard_similarity': 'Application layer',
    'fragmentation': 'Protocol layer',
    'betweenness': 'Physical layer',
})

In [14]:
# Create a plotly figure
fig = go.Figure()

for column in ['Application layer', 'Protocol layer', 'Physical layer']:
    fig.add_trace(go.Scatter(x=df['index'], y=df[column], name=column, opacity=0.6))

# Add a line for IFI with a thicker line
fig.add_trace(go.Scatter(x=df['index'], y=df['IFI'], name='IFI', line=dict(width=4)))

# Customize the layout
fig.update_layout(
    title="Internet Fragmentation Index",
    xaxis_title="Date",
    yaxis_title="Fragmentation"
)

# Show the plot
fig.show()

In [15]:
pyo.plot(fig, filename='ifi.html', auto_open=False)

'ifi.html'